# 第六讲  数据的载入与存储

In [8]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 1、文本格式数据的读写

将表格型数据读取为DataFrame对象是pandas的重要特性。

pd.read_csv(filename)    : 从csv文件中导入数据;  
pd.read_table(filename)  : 从限定分隔符的文本文件导入数据；  
pd.read_excel(filename)  : 从Excel文件导入数据；  
pd.read_json(json_string): 从JSON格式的字符串导入数据；  
pd.read_html(url)        : 解析URLL，字符串或者HTML文件；  
pd.read_sql(query,con)   : 从SQL表/库中导入数据；  
pd.read_clipboard()      : 从粘贴板获取内容；  


### （1）读取 CSV

* CSV:逗号分隔值（Comma-Separated Values，有时也称为字符分隔值，因为分隔字符也可以不是逗号），其文件以纯文本形式存储表格数据（数字和文本）。

In [9]:
# for windows
!type .\\examples\ex1.csv

# for Linux
#!cat /examples/ex1.csv   

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


* 使用read_csv函数

In [10]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 使用read_table函数, 并指定分隔符

In [11]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 读取没有表头的CSV

In [12]:
!type .\\examples\\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [13]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
pd.read_csv('examples/ex2.csv')


,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


* 指定列名

In [15]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


* 分层索引

In [17]:
!type .\\examples\csv_mindex.csv 

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

* 处理其他分割符

In [19]:
!type .\\examples\ex3.txt 

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [20]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [21]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


正则表达式\s表示匹配任何空白字符，包括空格、制表符、换页符等等, 等价于[ \f\n\r\t\v]  
而"\s+"则表示匹配任意多个上面的字符

In [22]:
!type .\\examples\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [23]:
pd.read_csv('examples/ex4.csv')

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [24]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 缺失值：（1）不显示 （2）使用标识值，如NA, NULL

In [25]:
!type .\\examples\ex5.csv


something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo
four,,13,15,16,
five,17


In [26]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1.0,2.0,3.0,4.0,NaN
1,two,5.0,6.0,NaN,8.0,world
2,three,9.0,10.0,11.0,12.0,foo
3,four,NaN,13.0,15.0,16.0,NaN
4,five,17.0,NaN,NaN,NaN,NaN


In [27]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False
3,False,True,False,False,False,True
4,False,False,True,True,True,True


* 指定缺失值

In [28]:
result = pd.read_csv('examples/ex5.csv', na_values=[5,'foo'])
result

,something,a,b,c,d,message
0,one,1.0,2.0,3.0,4.0,NaN
1,two,NaN,6.0,NaN,8.0,world
2,three,9.0,10.0,11.0,12.0,NaN
3,four,NaN,13.0,15.0,16.0,NaN
4,five,17.0,NaN,NaN,NaN,NaN


In [29]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1.0,2.0,3.0,4.0,NaN
1,NaN,5.0,6.0,NaN,8.0,world
2,three,9.0,10.0,11.0,12.0,NaN
3,four,NaN,13.0,15.0,16.0,NaN
4,five,17.0,NaN,NaN,NaN,NaN


* read_csv的参数多达50余个！！！

In [30]:
pd.read_csv?

In [31]:
pd.read_table?

### (2) 将数据写入文本文件

In [32]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1.0,2.0,3.0,4.0,NaN
1,two,5.0,6.0,NaN,8.0,world
2,three,9.0,10.0,11.0,12.0,foo
3,four,NaN,13.0,15.0,16.0,NaN
4,five,17.0,NaN,NaN,NaN,NaN


In [33]:
data.to_csv('examples/out.csv')

In [34]:
!type .\\examples\out.csv

,something,a,b,c,d,message
0,one,1.0,2.0,3.0,4.0,
1,two,5.0,6.0,,8.0,world
2,three,9.0,10.0,11.0,12.0,foo
3,four,,13.0,15.0,16.0,
4,five,17.0,,,,


使用其他分隔符

In [35]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1.0|2.0|3.0|4.0|
1|two|5.0|6.0||8.0|world
2|three|9.0|10.0|11.0|12.0|foo
3|four||13.0|15.0|16.0|
4|five|17.0||||


标识缺失值

In [36]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1.0,2.0,3.0,4.0,NULL
1,two,5.0,6.0,NULL,8.0,world
2,three,9.0,10.0,11.0,12.0,foo
3,four,NULL,13.0,15.0,16.0,NULL
4,five,17.0,NULL,NULL,NULL,NULL


禁止写入行、列标签

In [37]:
data.to_csv(sys.stdout, index=False, header=False)

one,1.0,2.0,3.0,4.0,
two,5.0,6.0,,8.0,world
three,9.0,10.0,11.0,12.0,foo
four,,13.0,15.0,16.0,
five,17.0,,,,


只写入子列，并按照指定的顺序

In [38]:
data.to_csv(sys.stdout, index=False, columns=['a', 'c', 'b'])

a,c,b
1.0,3.0,2.0
5.0,,6.0
9.0,11.0,10.0
,15.0,13.0
17.0,,


Series也有to_csv方法

In [39]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')

In [40]:
!type .\\examples\tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### (3) 使用分隔格式

In [41]:
!type .\\examples\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [42]:
pd.read_table('examples/ex7.csv')

,"a,""b"",""c"""
0,"1,""2"",""3"""
1,"1,""2"",""3"""


In [43]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [44]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [45]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
    print(lines)

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]


In [46]:
header, values = lines[0], lines[1:]

In [47]:
header

['a', 'b', 'c']

In [48]:
values

[['1', '2', '3'], ['1', '2', '3']]

In [49]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [50]:
f = open('examples/ex7.csv')
reader = csv.reader(f, delimiter = ',', quotechar = '"')

for line in reader:
    print(line)

f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [51]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

NameError: name 'my_dialect' is not defined

In [56]:
!type .\\mydata.csv

### (4) JSON数据

JSON(JavaScript Object Notation, JS 对象简谱) 是一种轻量级的数据交换格式。它基于 ECMAScript (欧洲计算机协会制定的js规范)的一个子集，采用完全独立于编程语言的文本格式来存储和表示数据。简洁和清晰的层次结构使得 JSON 成为理想的数据交换语言。 易于人阅读和编写，同时也易于机器解析和生成，并有效地提升网络传输效率。

In [57]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

* Python有好几库可用于读写JSON，例如json

使用json.loads函数将json读取为Python对象

In [58]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

也可以使用json.dumps函数将Python对象转换为JSON

In [59]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

自行决定如何将JSON对象转换为DataFrame或其他数据结构。  
一个比较方便的方式是将字典构成的列表传入DataFrame构造函数，并选出数据字段的子集。

In [60]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


* pandas.read_json函数可以自动将JSON数据集按照指定次序转换为Series或DataFrame.

In [ ]:
!type .\\examples\example.json

In [ ]:
data = pd.read_json('examples/example.json')
data

In [ ]:
!type .\\examples\ex1.json

In [ ]:
data = pd.read_json('examples/ex1.json')
data

pandas.read_json的默认选项是假设数组中的每个对象是表里的一行：

* 如果需要从pandas中将数据导出为JSON,一种办法是使用to_json方法

In [ ]:
print(data.to_json())
print(data.to_json(orient='records'))

###  (5) 读取和解析HTML中的表格

* Python拥有很多可以对HTML和XML格式进行读取、写入数据的库。例如lxml、Beautiful Soup和html5lib等

* pandas.read_html可以使用lxml和Beautiful Soup等库将HTML中的表自动解析为DataFrame对象。

In [ ]:
!conda install lxml
!pip install beautifulsoup4 html5lib

例子：美国银行倒闭数据HTML

In [ ]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)


In [ ]:
failures = tables[0]
failures

In [ ]:
failures.head()

计算每年倒闭的银行数量:

In [ ]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

In [ ]:
tables = pd.read_html('https://www.fdic.gov/buying/historical/loans/index.html')
len(tables)

In [ ]:
tables[2]

#### (6) 解析XML

* 例子：纽约大都会交通局火车和供给服务数据

<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

In [ ]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [ ]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

In [ ]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [ ]:
root
root.get('href')
root.text

##  2、二进制格式的读写

### (1) Pickle格式

* 使用Python内建的pickle序列化模块进行二进制格式操作是存储数据最高效、最方便的方式之一。

pandas对象拥有一个to_pickle方法可以将数据以pickle格式写入硬盘

In [ ]:
frame = pd.read_csv('examples/ex1.csv')
frame


In [ ]:
frame.to_pickle('examples/frame_pickle')

* 可以直接使用内建的pickle读取文件中pickle化的对象，也可以中央pandas.read_pickle来读取

In [ ]:
pd.read_pickle('examples/frame_pickle')

### （2）HDF5格式

 * HDF5：Hierarchical Data Format Version 5, 层次性数据格式第五版

* HDF5 三大要素：

hdf5 files： 能够存储两类数据对象 dataset 和 group 的容器，其操作类似 python 标准的文件操作；File 实例对象本身就是一个组，以 / 为名，是遍历文件的入口
dataset(array-like)： 可类比为 Numpy 数组，每个数据集都有一个名字（name）、形状（shape） 和类型（dtype），支持切片操作  
group(folder-like)： 可以类比为 字典，它是一种像文件夹一样的容器；group 中可以存放 dataset 或者其他的 group，键就是组成员的名称，值就是组成员对象本身(组或者数据集)


![avatar](HDF5.jpg)

![avatar](HDF5_2.png)

* 可以通过PyTables或h5py等库直接访问HDF5文件
* pandas也提供了高阶的接口，可以简化Series和DataFrame的存储

In [ ]:
frame = pd.DataFrame({'a': np.random.randn(100)})
frame

In [ ]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame

In [ ]:
store['obj1']

In [ ]:
store['obj1_col'] = frame['a']
store

* HDFStore支持两种存储模式： fixed 和 table

In [ ]:
frame2 = pd.DataFrame({'a': np.random.randn(100), 'b': np.random.randn(100)})
frame2

In [ ]:
store.put('obj2', frame2, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])

In [ ]:
store.close()

* 使用pandas.read_hdf方法

In [ ]:
frame2.to_hdf?

In [ ]:
frame2.to_hdf('mydata3.h5', 'obj3', format='table')


In [ ]:
frame_read = pd.read_hdf('mydata3.h5', 'obj3', where=['index < 5'])
frame_read

### （3）读取Excel

* pandas通过ExcelFile类 和 pandas.read_excel函数来读取Excel（2003及更高版本）格式的文件

* 使用 xlrd 和 openpyxl 附加包来分别读取XLS 和XLSX文件

In [ ]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

In [ ]:
pd.read_excel(xlsx, 'Sheet1')

In [ ]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

* 将数据写入Excel文件

In [ ]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

也可以直接将路径传入to_excel函数，避免显式生成ExcelWriter对象

In [ ]:
frame.to_excel('examples/ex2.xlsx')

## 3、与Web API交互

* 很多网站有公开的API，通过JSON或其他格式提供数据服务
* Python有多种方式来往返网站API。例如， requests包

例子：获取github上关于pandas的问题

In [52]:
import requests
import requests
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [403]>

In [53]:
data = resp.json()
data[0]['number']

KeyError: 0

In [54]:
data

{'message': "API rate limit exceeded for 138.113.10.20. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)",
 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}

In [55]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels','body', 'state'])
issues

,number,title,labels,body,state


## 4、与数据库交互

* 使用Python内建的sqlite3驱动来生成一个SQLite数据库

In [ ]:
import sqlite3

con = sqlite3.connect('mydata.sqlite')

query = """
Create table if not exists students
(id VARCHAR(16), name VARCHAR(32), gender INT, age INT, PRIMARY KEY (id) );
"""
try:
    con.execute(query)
    con.commit()
except:
    print('Create table failed！')

In [ ]:
data = [('202005001', 'Zhang San', 1, 21),
        ('202005002', 'Li Si',     0, 22),
        ('202005003', 'Wang Wu',   1, 22),
        ('202005004', 'Zhao Liu',  1, 22),
        ('202005005', 'Qian Qi',   0, 21),
        ('202005006', 'Sun Ba',    1, 22),
       ]

stmt = "INSERT INTO students VALUES(?, ?, ?, ?)"

try:
    con.executemany(stmt, data)
    con.commit()
except:
    print('Insert data failed!')

* 使用SQL读取

In [ ]:
cursor = con.execute('select * from students')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description

* 填入到DataFrame中

In [ ]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

*  还可以直接中的pandas的read_sql方法直接从SQLAlchemy连接中读取数据表

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')

try:
    df = pd.read_sql('select * from students where age>21', db)
except:
    print("Read data failed!")
    
df 